In [2]:
from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn import svm, metrics



ls = []
ls.extend(['N', 'L', 'R'])                    # N
ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
ls.extend(['V', 'E'])                         # VEB
ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]


ls2 = []
ls2.extend(["['N']","['L']", "['R']"])                    # N
ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
ls2.extend(["['V']", "['E']"])                         # VEB
ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])

rank = pd.read_csv("database/features_ranking.py").sort_values(['rfscore', 'features'], ascending=[0,1])

rank_n_f = pd.read_csv("database/features_n_vs_f_randomforest.py").sort_values(['rfscore', 'features'], ascending=[0,1])
rank_n_s = pd.read_csv("database/features_n_vs_s_randomforest.py").sort_values(['rfscore', 'features'], ascending=[0,1])
rank_n_v = pd.read_csv("database/features_n_vs_v_randomforest.py").sort_values(['rfscore', 'features'], ascending=[0,1])
rank_s_f = pd.read_csv("database/features_s_vs_f_randomforest.py").sort_values(['rfscore', 'features'], ascending=[0,1])
rank_s_v = pd.read_csv("database/features_s_vs_v_randomforest.py").sort_values(['rfscore', 'features'], ascending=[0,1])
rank_v_f = pd.read_csv("database/features_v_vs_f_randomforest.py").sort_values(['rfscore', 'features'], ascending=[0,1])







import pickle
f = open('database/nvs_lin.text', 'rb')
n_s_ln = pickle.load(f)[1]
f.close()

f = open('database/nvv_lin.text', 'rb')
n_v_ln = pickle.load(f)[1]
f.close()

f = open('database/nvf_lin.text', 'rb')
n_f_ln = pickle.load(f)[1]
f.close()

f = open('database/svv_lin.text', 'rb')
s_v_ln = pickle.load(f)[1]
f.close()

f = open('database/svf_lin.text', 'rb')
s_f_ln = pickle.load(f)[1]
f.close()

f = open('database/vvf_lin.text', 'rb')
v_f_ln = pickle.load(f)[1]
f.close()

import pickle
f = open('database/nvs_log.text', 'rb')
n_s_log = pickle.load(f)[1]
f.close()

f = open('database/nvv_log.text', 'rb')
n_v_log = pickle.load(f)[1]
f.close()

f = open('database/nvf_log.text', 'rb')
n_f_log = pickle.load(f)[1]
f.close()

f = open('database/svv_log.text', 'rb')
s_v_log = pickle.load(f)[1]
f.close()

f = open('database/svf_log.text', 'rb')
s_f_log = pickle.load(f)[1]
f.close()

f = open('database/vvf_log.text', 'rb')
v_f_log = pickle.load(f)[1]
f.close()



score = 0.01
score_n_s = 0.01
score_n_f = 0.015
score_n_v = 0.01
score_s_f = 0.01
score_s_v = 0.015
score_v_f = 0.01
feature_good = rank[rank['rfscore'] >= score]['features'].values
n_s = rank_n_s[rank_n_s['rfscore'] >= score_n_s]['features'].values
n_f = rank_n_f[rank_n_f['rfscore'] >= score_n_f]['features'].values
n_v = rank_n_v[rank_n_v['rfscore'] >= score_n_v]['features'].values
s_f = rank_s_f[rank_s_f['rfscore'] >= score_s_f]['features'].values
s_v = rank_s_v[rank_s_v['rfscore'] >= score_s_v]['features'].values
v_f = rank_v_f[rank_v_f['rfscore'] >= score_v_f]['features'].values

In [3]:
np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )


In [4]:
np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]
features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()

#X_train_under = pd.DataFrame(under_X_train,columns=features_clinic)
X_train_balanced = pd.DataFrame(good_features_X,columns=features_clinic)
X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)

In [5]:
#classifier.logisticRegress(X_train[new_row], y_train, X_test[new_row], y_test, jk=True,labels=[0,1,2,3])[1]
#y_train[y_train == 0] = 0
#y_train[y_train != 0] = 1

#y_test[y_test == 0] = 0
#y_test[y_test != 0] = 1
print("y")
y = classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])[2]
print("y1")
y1 = classifier.logisticRegress(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])[2]
print("y2")
y2 = classifier.Linear_D(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])[2]

print("y3")
y3 = classifier.logisticRegress(X_train[n_s_ln], y_train, X_test[n_s_ln], y_test, jk=False,labels=[0,1,2,3])[2]
print("y4")
y4 = classifier.Linear_D(X_train[n_s_ln], y_train, X_test[n_s_ln], y_test, jk=False,labels=[0,1,2,3])[2]
print("y5")
y5 = classifier.Linear_D(X_train[n_v_ln], y_train, X_test[n_v_ln], y_test, jk=False,labels=[0,1,2,3])[2]


print("y6")
y6 = classifier.Linear_D(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])[2]
print("y7")
y7 = classifier.logisticRegress(X_train[n_v_log], y_train, X_test[n_v_log], y_test, jk=False,labels=[0,1,2,3])[2]
print("y8")
y8 = classifier.logisticRegress(X_train[v_f_log], y_train, X_test[v_f_log], y_test, jk=False,labels=[0,1,2,3])[2]
print("y9")
y9 = classifier.Linear_D(X_train[n_v_log], y_train, X_test[n_v_log], y_test, jk=False,labels=[0,1,2,3])[2]
print("y10")
y10 = classifier.Linear_D(X_train[v_f_log], y_train, X_test[v_f_log], y_test, jk=False,labels=[0,1,2,3])[2]
print("y11")
y11 = classifier.Linear_D(X_train[n_v_log], y_train, X_test[n_v_ln], y_test, jk=False,labels=[0,1,2,3])[2]



#classifier.voting_ensemble_lin_rf(X_train[new_row], y_train, X_test[new_row], y_test, jk=True,labels=[0,1,2,3])
#classifier.logisticRegress(X_train[rank], y_train, X_test[rank], y_test, jk=True,labels=[0,1,2,3])
#classifier.Linear_D(X_train[rank], y_train, X_test[rank], y_test, jk=True,labels=[0,1,2,3])
y

y
[[41468  1081  1090   382]
 [  457  1402   186     5]
 [  411   377  2334    98]
 [  130     1    22   235]]
              precision    recall  f1-score   support

           0       0.98      0.94      0.96     44021
           1       0.49      0.68      0.57      2050
           2       0.64      0.72      0.68      3220
           3       0.33      0.61      0.42       388

   micro avg       0.91      0.91      0.91     49679
   macro avg       0.61      0.74      0.66     49679
weighted avg       0.93      0.91      0.92     49679

y1
[[42752   373   525   371]
 [ 1508   111   402    29]
 [  178   165  2620   257]
 [  232     1     3   152]]
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     44021
           1       0.17      0.05      0.08      2050
           2       0.74      0.81      0.77      3220
           3       0.19      0.39      0.25       388

   micro avg       0.92      0.92      0.92     49679
   macro avg    

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


[[41176   386  1658   801]
 [ 1737   262    51     0]
 [  838   318  1874   190]
 [   42     0    40   306]]
              precision    recall  f1-score   support

           0       0.94      0.94      0.94     44021
           1       0.27      0.13      0.17      2050
           2       0.52      0.58      0.55      3220
           3       0.24      0.79      0.36       388

   micro avg       0.88      0.88      0.88     49679
   macro avg       0.49      0.61      0.51     49679
weighted avg       0.88      0.88      0.88     49679

y3
[[41034   955  1021  1011]
 [  512  1245   237    56]
 [  537   404  2226    53]
 [  361     0     6    21]]
              precision    recall  f1-score   support

           0       0.97      0.93      0.95     44021
           1       0.48      0.61      0.54      2050
           2       0.64      0.69      0.66      3220
           3       0.02      0.05      0.03       388

   micro avg       0.90      0.90      0.90     49679
   macro avg      

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


[[42710   778   416   117]
 [ 1076   857   117     0]
 [ 1866   359   939    56]
 [  360     0    16    12]]
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     44021
           1       0.43      0.42      0.42      2050
           2       0.63      0.29      0.40      3220
           3       0.06      0.03      0.04       388

   micro avg       0.90      0.90      0.90     49679
   macro avg       0.51      0.43      0.45     49679
weighted avg       0.88      0.90      0.88     49679

y5
[[43521   258   212    30]
 [ 2019    14    16     1]
 [  764    57  2205   194]
 [  349    12     5    22]]
              precision    recall  f1-score   support

           0       0.93      0.99      0.96     44021
           1       0.04      0.01      0.01      2050
           2       0.90      0.68      0.78      3220
           3       0.09      0.06      0.07       388

   micro avg       0.92      0.92      0.92     49679
   macro avg      

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


[[43556   152   286    27]
 [ 1633   263   154     0]
 [  588    82  2341   209]
 [  287     0     2    99]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     44021
           1       0.53      0.13      0.21      2050
           2       0.84      0.73      0.78      3220
           3       0.30      0.26      0.27       388

   micro avg       0.93      0.93      0.93     49679
   macro avg       0.65      0.52      0.56     49679
weighted avg       0.92      0.93      0.92     49679

y7
[[42258   511   840   412]
 [  963    96   924    67]
 [  645   130  2287   158]
 [  380     0     7     1]]
              precision    recall  f1-score   support

           0       0.96      0.96      0.96     44021
           1       0.13      0.05      0.07      2050
           2       0.56      0.71      0.63      3220
           3       0.00      0.00      0.00       388

   micro avg       0.90      0.90      0.90     49679
   macro avg      

array([1, 0, 0, ..., 0, 0, 0])

In [187]:
np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]
features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()

#X_train_under = pd.DataFrame(under_X_train,columns=features_clinic)
X_train_balanced = pd.DataFrame(good_features_X,columns=features_clinic)
X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)

y_train[y_train != 2] = 0
y_train[y_train == 2] = 1

y_test[y_test != 2] = 0
y_test[y_test == 2] = 1

print("y")
y = classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])[2]
print("y1")
y1 = classifier.logisticRegress(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])[2]
print("y2")
y2 = classifier.Linear_D(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])[2]

print("y3")
y3 = classifier.logisticRegress(X_train[n_s_ln], y_train, X_test[n_s_ln], y_test, jk=False,labels=[0,1,2,3])[2]
print("y4")
y4 = classifier.Linear_D(X_train[n_s_ln], y_train, X_test[n_s_ln], y_test, jk=False,labels=[0,1,2,3])[2]
print("y5")
y5 = classifier.Linear_D(X_train[n_v_ln], y_train, X_test[n_v_ln], y_test, jk=False,labels=[0,1,2,3])[2]


print("y6")
y6 = classifier.Linear_D(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])[2]
print("y7")
y7 = classifier.logisticRegress(X_train[n_v_log], y_train, X_test[n_v_log], y_test, jk=False,labels=[0,1,2,3])[2]
print("y8")
y8 = classifier.logisticRegress(X_train[v_f_log], y_train, X_test[v_f_log], y_test, jk=False,labels=[0,1,2,3])[2]
print("y9")
y9 = classifier.Linear_D(X_train[n_v_log], y_train, X_test[n_v_log], y_test, jk=False,labels=[0,1,2,3])[2]
print("y10")
y10 = classifier.Linear_D(X_train[v_f_log], y_train, X_test[v_f_log], y_test, jk=False,labels=[0,1,2,3])[2]
print("y11")
y11 = classifier.Linear_D(X_train[n_v_log], y_train, X_test[n_v_ln], y_test, jk=False,labels=[0,1,2,3])[2]



y
[[41342  5117     0     0]
 [  372  2848     0     0]
 [    0     0     0     0]
 [    0     0     0     0]]
              precision    recall  f1-score   support

           0       0.99      0.89      0.94     46459
           1       0.36      0.88      0.51      3220
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0

   micro avg       0.89      0.89      0.89     49679
   macro avg       0.34      0.44      0.36     49679
weighted avg       0.95      0.89      0.91     49679

y1


C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[[44262  2197     0     0]
 [  223  2997     0     0]
 [    0     0     0     0]
 [    0     0     0     0]]
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     46459
           1       0.58      0.93      0.71      3220
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0

   micro avg       0.95      0.95      0.95     49679
   macro avg       0.39      0.47      0.42     49679
weighted avg       0.97      0.95      0.96     49679

y2


C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


[[44203  2256     0     0]
 [ 1128  2092     0     0]
 [    0     0     0     0]
 [    0     0     0     0]]
              precision    recall  f1-score   support

           0       0.98      0.95      0.96     46459
           1       0.48      0.65      0.55      3220
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0

   micro avg       0.93      0.93      0.93     49679
   macro avg       0.36      0.40      0.38     49679
weighted avg       0.94      0.93      0.94     49679

y3


...

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[[45932   527     0     0]
 [  773  2447     0     0]
 [    0     0     0     0]
 [    0     0     0     0]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     46459
           1       0.82      0.76      0.79      3220
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0

   micro avg       0.97      0.97      0.97     49679
   macro avg       0.45      0.44      0.44     49679
weighted avg       0.97      0.97      0.97     49679

y11


C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[[46250   209     0     0]
 [ 2990   230     0     0]
 [    0     0     0     0]
 [    0     0     0     0]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     46459
           1       0.52      0.07      0.13      3220
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0

   micro avg       0.94      0.94      0.94     49679
   macro avg       0.37      0.27      0.27     49679
weighted avg       0.91      0.94      0.91     49679



C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


y


NameError: name 'X_train_under' is not defined

[[40941    45   128  2907]
 [ 1994    15     8    33]
 [  272    34  2767   147]
 [   73     0     8   307]]
              precision    recall  f1-score   support

           0       0.95      0.93      0.94     44021
           1       0.16      0.01      0.01      2050
           2       0.95      0.86      0.90      3220
           3       0.09      0.79      0.16       388

   micro avg       0.89      0.89      0.89     49679
   macro avg       0.54      0.65      0.50     49679
weighted avg       0.91      0.89      0.89     49679



## V class

In [1]:
#### For v class

#y_test[y_test != 2] = 0
#y_test[y_test == 2] = 1
y_train_b = np.copy(y_train)
y_train_b[y_train_b == 3] = -1

y_train_b[y_train_b == 0] = -1
y_train_b[y_train_b >= 0] = 0

y_train_b[y_train_b < 0] = 1
#y_train_b[y_train_b == 2] = 1

y=classifier.randomForest(X_train[v_f_log], y_train_b, X_test[v_f_log], y_test, jk=False,labels=[0,1,2,3])[2]
y5=classifier.Linear_D(X_train[n_f_log], y_train_b, X_test[n_f_log], y_test, jk=False,labels=[0,1,2,3])[2]
y6=classifier.logisticRegress(X_train[n_f_log], y_train_b, X_test[n_f_log], y_test, jk=False,labels=[0,1,2,3])[2]




NameError: name 'np' is not defined

In [16]:
y_1 = y[y==1]

y_test_1 = y_test[y==1]

X_test_1 = X_test[y==1]

y_train_b = np.copy(y_train)
y_train_b[y_train_b == 2] = 1

y_train_b[y_train_b != 2] = 0

y2=classifier.randomForest(X_train[v_f_log], y_train_b, X_test_1[v_f_log], y_test_1, jk=False,labels=[0,1,2,3])[2]
y3=classifier.Linear_D(X_train[n_f_log], y_train_b, X_test_1[n_f_log], y_test_1, jk=False,labels=[0,1,2,3])[2]
y4=classifier.logisticRegress(X_train[n_f_log], y_train_b, X_test_1[n_f_log], y_test_1, jk=False,labels=[0,1,2,3])[2]


[[4015    0    0    0]
 [ 545    0    0    0]
 [2970    0    0    0]
 [ 358    0    0    0]]
              precision    recall  f1-score   support

           0       0.51      1.00      0.67      4015
           1       0.00      0.00      0.00       545
           2       0.00      0.00      0.00      2970
           3       0.00      0.00      0.00       358

   micro avg       0.51      0.51      0.51      7888
   macro avg       0.13      0.25      0.17      7888
weighted avg       0.26      0.51      0.34      7888



C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[4015    0    0    0]
 [ 545    0    0    0]
 [2970    0    0    0]
 [ 358    0    0    0]]
              precision    recall  f1-score   support

           0       0.51      1.00      0.67      4015
           1       0.00      0.00      0.00       545
           2       0.00      0.00      0.00      2970
           3       0.00      0.00      0.00       358

   micro avg       0.51      0.51      0.51      7888
   macro avg       0.13      0.25      0.17      7888
weighted avg       0.26      0.51      0.34      7888



C:\Users\chont\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:402: RuntimeWarning: invalid value encountered in true_divide
  S**2))[:self._max_components]
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

In [8]:


y=classifier.svm_model_linear(X_train[v_f_log], y_train_b, X_test[v_f_log], y_test, jk=False,labels=[0,1,2,3])[2]
y7=classifier.Linear_D(X_train[n_f_log], y_train_b, X_test[n_f_log], y_test, jk=False,labels=[0,1,2,3])[2]
y6=classifier.xgboost(X_train[n_f_log], y_train_b, X_test[n_f_log], y_test, jk=False,labels=[0,1,2,3])[2]


[[43849   172     0     0]
 [ 1861   189     0     0]
 [  918  2302     0     0]
 [  377    11     0     0]]
              precision    recall  f1-score   support

           0       0.93      1.00      0.96     44021
           1       0.07      0.09      0.08      2050
           2       0.00      0.00      0.00      3220
           3       0.00      0.00      0.00       388

   micro avg       0.89      0.89      0.89     49679
   macro avg       0.25      0.27      0.26     49679
weighted avg       0.83      0.89      0.86     49679



C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[42922  1099     0     0]
 [ 2013    37     0     0]
 [  412  2808     0     0]
 [  167   221     0     0]]
              precision    recall  f1-score   support

           0       0.94      0.98      0.96     44021
           1       0.01      0.02      0.01      2050
           2       0.00      0.00      0.00      3220
           3       0.00      0.00      0.00       388

   micro avg       0.86      0.86      0.86     49679
   macro avg       0.24      0.25      0.24     49679
weighted avg       0.84      0.86      0.85     49679



C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[40398  3623     0     0]
 [ 2021    29     0     0]
 [  415  2805     0     0]
 [  182   206     0     0]]
              precision    recall  f1-score   support

           0       0.94      0.92      0.93     44021
           1       0.00      0.01      0.01      2050
           2       0.00      0.00      0.00      3220
           3       0.00      0.00      0.00       388

   micro avg       0.81      0.81      0.81     49679
   macro avg       0.24      0.23      0.23     49679
weighted avg       0.83      0.81      0.82     49679



[[45739   720     0     0]
 [  443  2777     0     0]
 [    0     0     0     0]
 [    0     0     0     0]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     46459
           1       0.79      0.86      0.83      3220
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0

   micro avg       0.98      0.98      0.98     49679
   macro avg       0.45      0.46      0.45     49679
weighted avg       0.98      0.98      0.98     49679



C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[[41395  5064     0     0]
 [  175  3045     0     0]
 [    0     0     0     0]
 [    0     0     0     0]]
              precision    recall  f1-score   support

           0       1.00      0.89      0.94     46459
           1       0.38      0.95      0.54      3220
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0

   micro avg       0.89      0.89      0.89     49679
   macro avg       0.34      0.46      0.37     49679
weighted avg       0.96      0.89      0.91     49679



C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [195]:
#y_test[y_test != 2] = 0
#y_test[y_test == 2] = 1

#y_train[y_train != 2] = 0
#y_train[y_train == 2] = 1

from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
scale_test = StandardScaler()
scale.fit(X_train)
scale_test.fit(X_test)
X_train_s = scale.transform(X_train)
X_test_s = scale_test.transform(X_test)








In [196]:
y7=classifier.Linear_D(X_train_s[n_f_log], y_train, X_test_s[n_f_log], y_test, jk=False)[2]
y6=classifier.logisticRegress(X_train[n_f_log], y_train, X_test[n_f_log], y_test, jk=False)[2]

[[39761    21   123  4116]
 [ 2004     1     5    40]
 [  343    80  2420   377]
 [   23     0     6   359]]
              precision    recall  f1-score   support

           0       0.94      0.90      0.92     44021
           1       0.01      0.00      0.00      2050
           2       0.95      0.75      0.84      3220
           3       0.07      0.93      0.14       388

   micro avg       0.86      0.86      0.86     49679
   macro avg       0.49      0.65      0.47     49679
weighted avg       0.90      0.86      0.87     49679

[[39880   850   960  2331]
 [ 1837   160    14    39]
 [  214    55  2799   152]
 [  139     1    45   203]]
              precision    recall  f1-score   support

           0       0.95      0.91      0.93     44021
           1       0.15      0.08      0.10      2050
           2       0.73      0.87      0.80      3220
           3       0.07      0.52      0.13       388

   micro avg       0.87      0.87      0.87     49679
   macro avg       0.

In [14]:
inds1 = []
for i in range(len(y)):
    #avg.append(np.mean([y[i], y1[i], y2[i], y3[i], y4[i], y4[i],y5[i], y6[i], y7[i], y8[i]]))
    #avg.append(round(np.mean([y[i],y3[i], y4[i],  y5[i],  y6[i]]),0))
    #print(np.mean(  [y5[i],  y6[i]]))
   # indes.append(voting( [y5[i],y4[i],  y6[i]])
    #inds.append(voting( [y[i],y3[i],  y6[i], y7[i]], focus = 2, amount=2))
    #inds.append(voting( [y[i],y3[i],  y6[i]], focus = 2, amount=2))
    #if(2 in [y[i],y6[i],  y7[i]]):
    inds1.append(classifier.voting( [y[i],y6[i],  y7[i]], focus = 2, amount=2, random = False))



print(metrics.confusion_matrix(y_test,inds1 , labels=[0,1,2,3]))
print(classification_report(y_test,inds1))

[[40903    48   127  2943]
 [ 1994    15     8    33]
 [  272    34  2767   147]
 [   72     0     7   309]]
              precision    recall  f1-score   support

           0       0.95      0.93      0.94     44021
           1       0.15      0.01      0.01      2050
           2       0.95      0.86      0.90      3220
           3       0.09      0.80      0.16       388

   micro avg       0.89      0.89      0.89     49679
   macro avg       0.54      0.65      0.50     49679
weighted avg       0.91      0.89      0.89     49679



In [15]:
inds1 = np.asarray(inds1)
X_test_s = X_test[inds1!=2]
y_test_s = y_test[inds1!=2]


X_train_s = X_train[y_train != 2]
y_train_s = y_train[y_train != 2]

y_pred_v = inds1[inds1==2]
y_true_v = y_test[inds1==2]

## S class

In [72]:

y2 = classifier.logisticRegress(X_train[n_s], y_train, X_test_s[n_s], y_test_s, jk=False,labels=[0,1,2,3])[2]
y5 = classifier.logisticRegress(X_train[n_s_ln], y_train, X_test_s[n_s_ln], y_test_s, jk=False,labels=[0,1,2,3])[2]

inds2 = []
for i in range(len(y2)):
    #avg.append(np.mean([y[i], y1[i], y2[i], y3[i], y4[i], y4[i],y5[i], y6[i], y7[i], y8[i]]))
    #avg.append(round(np.mean([y[i],y3[i], y4[i],  y5[i],  y6[i]]),0))
    #print(np.mean(  [y5[i],  y6[i]]))
   # indes.append(voting( [y5[i],y4[i],  y6[i]])
    #inds.append(voting( [y[i],y3[i],  y6[i], y7[i]], focus = 2, amount=2))
    #inds.append(voting( [y[i],y3[i],  y6[i]], focus = 2, amount=2))
    inds2.append(classifier.voting( [y2[i],y5[i]], focus = 1, amount=1, random = True))

    



print(metrics.confusion_matrix(y_test_s,inds2 , labels=[0,1,2,3]))
print(classification_report(y_test_s,inds2))

[[41397  1068  1051   378]
 [  457  1396   184     5]
 [  147    68   180    58]
 [  128     1    18   234]]
              precision    recall  f1-score   support

           0       0.98      0.94      0.96     43894
           1       0.55      0.68      0.61      2042
           2       0.13      0.40      0.19       453
           3       0.35      0.61      0.44       381

   micro avg       0.92      0.92      0.92     46770
   macro avg       0.50      0.66      0.55     46770
weighted avg       0.95      0.92      0.94     46770

[[40956   946   981  1011]
 [  511  1243   232    56]
 [  213    76   146    18]
 [  358     0     3    20]]
              precision    recall  f1-score   support

           0       0.97      0.93      0.95     43894
           1       0.55      0.61      0.58      2042
           2       0.11      0.32      0.16       453
           3       0.02      0.05      0.03       381

   micro avg       0.91      0.91      0.91     46770
   macro avg       0.

In [75]:
inds2 = np.asarray(inds2)
X_test_f = X_test_s[inds2!=1]
y_test_f = y_test_s[inds2!=1]


X_train_f = X_train_s[y_train_s != 1]
y_train_f = y_train_s[y_train_s != 1]

y_pred_s = inds[inds2==1]
y_true_s = y_test_s[inds2==1]

In [76]:
f2 = classifier.Linear_D(X_train[n_s], y_train, X_test_f[n_s], y_test_f, jk=False,labels=[0,1,2,3])[2]
f3 = classifier.logisticRegress(X_train[n_s], y_train, X_test_f[n_s], y_test_f, jk=False,labels=[0,1,2,3])[2]
f4 = classifier.Linear_D(X_train[v_f_log], y_train, X_test_f[v_f_log], y_test_f, jk=False,labels=[0,1,2,3])[2]
f5 = classifier.logisticRegress(X_train[n_f_ln], y_train, X_test_f[n_f_ln], y_test_f, jk=True,labels=[0,1,2,3])[2]
f6= classifier.Linear_D(X_train[v_f_ln], y_train, X_test_f[v_f_ln], y_test_f, jk=True,labels=[0,1,2,3])[2]
f7=classifier.Linear_D(X_train[s_f_log], y_train, X_test_f[s_f_log], y_test_f, jk=True,labels=[0,1,2,3])[2]
f8=classifier.Linear_D(X_train[v_f_log], y_train, X_test_f[v_f_log], y_test_f, jk=True,labels=[0,1,2,3])[2]


C:\Users\chont\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


[[40160     6  1418   798]
 [  449     2    38     0]
 [  173     6    64    84]
 [   38     0    37   305]]
              precision    recall  f1-score   support

           0       0.98      0.95      0.97     42382
           1       0.14      0.00      0.01       489
           2       0.04      0.20      0.07       327
           3       0.26      0.80      0.39       380

   micro avg       0.93      0.93      0.93     43578
   macro avg       0.36      0.49      0.36     43578
weighted avg       0.96      0.93      0.94     43578

[[41170     0   834   378]
 [  414     0    70     5]
 [  131     0   140    56]
 [  128     0    18   234]]
              precision    recall  f1-score   support

           0       0.98      0.97      0.98     42382
           1       0.00      0.00      0.00       489
           2       0.13      0.43      0.20       327
           3       0.35      0.62      0.44       380

   micro avg       0.95      0.95      0.95     43578
   macro avg       0.

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[41709    12   148   513]
 [  458     3    17    11]
 [  177     6    55    89]
 [   42     0     0   338]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     42382
           1       0.14      0.01      0.01       489
           2       0.25      0.17      0.20       327
           3       0.36      0.89      0.51       380

   micro avg       0.97      0.97      0.97     43578
   macro avg       0.43      0.51      0.43     43578
weighted avg       0.96      0.97      0.96     43578

[[36080   255  2443  3604]
 [  265    19   165    40]
 [  148     5    33   141]
 [   22     0     3   355]]
              precision    recall  f1-score   support

           0       0.99      0.85      0.91     42382
           1       0.07      0.04      0.05       489
           2       0.01      0.10      0.02       327
           3       0.09      0.93      0.16       380

   micro avg       0.84      0.84      0.84     43578
   macro avg       0.

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


[[39600     6   635  2141]
 [  468     2    11     8]
 [  168     0    76    83]
 [   29     0     1   350]]
              precision    recall  f1-score   support

           0       0.98      0.93      0.96     42382
           1       0.25      0.00      0.01       489
           2       0.11      0.23      0.14       327
           3       0.14      0.92      0.24       380

   micro avg       0.92      0.92      0.92     43578
   macro avg       0.37      0.52      0.34     43578
weighted avg       0.96      0.92      0.94     43578

[0.19135216807294642, 0.5916234473892537, 0.16962901496012991]


C:\Users\chont\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


[[41442    12   361   567]
 [  475     0     6     8]
 [  142     1   104    80]
 [   33     0     4   343]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     42382
           1       0.00      0.00      0.00       489
           2       0.22      0.32      0.26       327
           3       0.34      0.90      0.50       380

   micro avg       0.96      0.96      0.96     43578
   macro avg       0.39      0.55      0.43     43578
weighted avg       0.96      0.96      0.96     43578

[0.35750248402516854, 0.5369901818767101, 0.24587501474717305]
[[41709    12   148   513]
 [  458     3    17    11]
 [  177     6    55    89]
 [   42     0     0   338]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     42382
           1       0.14      0.01      0.01       489
           2       0.25      0.17      0.20       327
           3       0.36      0.89      0.51       380

   micro avg

## F class

In [77]:
inds3 = []
for i in range(len(f2)):
    #avg.append(np.mean([y[i], y1[i], y2[i], y3[i], y4[i], y4[i],y5[i], y6[i], y7[i], y8[i]]))
    #avg.append(round(np.mean([y[i],y3[i], y4[i],  y5[i],  y6[i]]),0))
    #print(np.mean(  [y5[i],  y6[i]]))
   # indes.append(voting( [y5[i],y4[i],  y6[i]])
    #inds.append(voting( [y[i],y3[i],  y6[i], y7[i]], focus = 2, amount=2))
    #inds.append(voting( [y[i],y3[i],  y6[i]], focus = 2, amount=2))
    #print(classifier.voting( [f2[i],f3[i], f4[i], f5[i], f6[i], f7[i], f8[i]], focus = 3, amount=3, random = True))
#
   # inds3.append(classifier.voting( [f2[i],f3[i], f4[i], f5[i], f6[i], f7[i], f8[i]], focus = 3, amount=3, random = True))
    inds3.append(classifier.voting( [ f4[i], f6[i], f7[i], f8[i]], focus = 3, amount=2, random = True))


    
    

print(metrics.confusion_matrix(y_test_f,f7 , labels=[0,1,2,3]))
print(classification_report(y_test_f,f7))

[[41442    12   361   567]
 [  475     0     6     8]
 [  142     1   104    80]
 [   33     0     4   343]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     42382
           1       0.00      0.00      0.00       489
           2       0.22      0.32      0.26       327
           3       0.34      0.90      0.50       380

   micro avg       0.96      0.96      0.96     43578
   macro avg       0.39      0.55      0.43     43578
weighted avg       0.96      0.96      0.96     43578



In [78]:
#y_pred_v = inds1[inds1==2]
#y_true_v = y_test[inds1==2]

#y_pred_s = inds[inds2==1]
#y_true_s = y_test_s[inds2==1]
y_pred_f = f7
f7[f7 != 3] = 0
y_test_f

array([0, 0, 0, ..., 0, 0, 0])

In [79]:

y_pred = np.hstack((y_pred_v, y_pred_s , y_pred_f))
y_true = np.hstack((y_true_v,y_true_s,  y_test_f))
y_pred_n = np.hstack((y_pred_v, y_pred_s , f7))


## Overall results

In [80]:


print(metrics.confusion_matrix(y_true,y_pred , labels=[0,1,2,3]))
print(classification_report(y_true,y_pred))


print(metrics.confusion_matrix(y_true,y_pred_n , labels=[0,1,2,3]))
print(classification_report(y_true,y_pred_n))

[[41815  1512   127   567]
 [  481  1553     8     8]
 [  247   126  2767    80]
 [   37     1     7   343]]
              precision    recall  f1-score   support

           0       0.98      0.95      0.97     44021
           1       0.49      0.76      0.59      2050
           2       0.95      0.86      0.90      3220
           3       0.34      0.88      0.49       388

   micro avg       0.94      0.94      0.94     49679
   macro avg       0.69      0.86      0.74     49679
weighted avg       0.95      0.94      0.94     49679

[[41815  1512   127   567]
 [  481  1553     8     8]
 [  247   126  2767    80]
 [   37     1     7   343]]
              precision    recall  f1-score   support

           0       0.98      0.95      0.97     44021
           1       0.49      0.76      0.59      2050
           2       0.95      0.86      0.90      3220
           3       0.34      0.88      0.49       388

   micro avg       0.94      0.94      0.94     49679
   macro avg       0.

In [81]:
labels = [0,1,2,3]
print(metric.get_metrics(y_true,y_pred,lb=labels))
print(metric.get_metrics(y_true,y_pred_n,lb=labels))


[0.7245367357215096, 3.0545925424129345, 0.7440924356623716]
[0.7245367357215096, 3.0545925424129345, 0.7440924356623716]


## Random projection

In [114]:
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection

In [139]:
transformer = SparseRandomProjection(n_components = 10, eps=0.1)
transformer.fit(X_train[n_s])
X_new = transformer.transform(X_train[n_s])
X_new_test = transformer.transform(X_test[n_s])

transformer = GaussianRandomProjection(n_components = 10, eps=0.1)
transformer.fit(X_train[n_s])
X_new_g = transformer.transform(X_train[n_s])
X_new_test_g = transformer.transform(X_test[n_s])

In [147]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(X_new_g)


In [140]:
classifier.logisticRegress(X_new, y_train, X_new_test, y_test, jk=True,labels=[0,1,2,3])[2]
classifier.logisticRegress(X_new_g, y_train, X_new_test_g, y_test, jk=True,labels=[0,1,2,3])[2]

classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=True,labels=[0,1,2,3])[2]


[[41297  1096  1052   576]
 [  433  1501   109     7]
 [  360   308  2461    91]
 [  114     1    25   248]]
              precision    recall  f1-score   support

           0       0.98      0.94      0.96     44021
           1       0.52      0.73      0.61      2050
           2       0.67      0.76      0.72      3220
           3       0.27      0.64      0.38       388

   micro avg       0.92      0.92      0.92     49679
   macro avg       0.61      0.77      0.66     49679
weighted avg       0.93      0.92      0.92     49679

[0.6499460300609754, 2.6877995926047715, 0.6609479641060841]
[[41385  1094   924   618]
 [  457  1487    94    12]
 [  418   355  2351    96]
 [  127     1    25   235]]
              precision    recall  f1-score   support

           0       0.98      0.94      0.96     44021
           1       0.51      0.73      0.60      2050
           2       0.69      0.73      0.71      3220
           3       0.24      0.61      0.35       388

   micro avg  

array([1, 0, 0, ..., 0, 0, 0])

In [ ]:
transformer = GaussianRandomProjection(n_components = 10, eps=0.1)
transformer.fit(X_train[n_s])
X_new = transformer.transform(X_train[n_s])
X_new_test = transformer.transform(X_test[n_s])

In [182]:
# rr, HOS, myMorph training features
train_rr=np.loadtxt('database/rr-train.txt', dtype=float)[:,1:] # extract class labels
train_hos=np.loadtxt('database/hos_train.txt', dtype=float)
train_mmorph=np.loadtxt('database/morph_train.txt', dtype=float)

# rr, HOS, myMorph testing features
test_rr=np.loadtxt('database/rr-test.txt', dtype=float)[:,1:] # extract class labels
test_rr=np.loadtxt('database/rr-test.txt', dtype=float)[:,1:] # extract class labels
test_hos=np.loadtxt('database/hos_test.txt', dtype=float)
test_mmorph=np.loadtxt('database/morph_test.txt', dtype=float)

# training and testing class labels.
train_labels=np.loadtxt('database/labels_train.txt', dtype=str)
test_labels=np.loadtxt('database/labels_test.txt', dtype=str)

# aggregate all features.
train=np.hstack((train_rr, train_mmorph))
test=np.hstack((test_rr, test_mmorph))



In [151]:
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection

transformer = GaussianRandomProjection(n_components = 2, eps=0.1)
transformer.fit(train_rr[:,3:8])
train_g = transformer.transform(train_rr[:,3:8])
test_g = transformer.transform(test_rr[:,3:8])

In [185]:

train_labels[train_labels != 'V'] = 0
train_labels[train_labels == 'V'] = 1

test_labels[test_labels != 'V'] = 0
test_labels[test_labels == 'V'] = 1

#classifier.Linear_D(train_g, train_labels, test_g, test_labels, jk=False,labels=[0,1])[2]
classifier.Linear_D(train, train_labels, test, test_labels, jk=False,labels=['0','1'])[2]


[[46059   398]
 [  980  2240]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     46457
           1       0.85      0.70      0.76      3220

   micro avg       0.97      0.97      0.97     49677
   macro avg       0.91      0.84      0.88     49677
weighted avg       0.97      0.97      0.97     49677



array(['0', '0', '0', ..., '0', '0', '0'], dtype='<U1')

In [183]:
classifier.Linear_D(train, train_labels, test, test_labels, jk=False,labels=['0','1'])[2]


ValueError: At least one label specified must be in y_true

In [184]:
classifier.Linear_D(train, train_labels, test, test_labels, jk=False,labels=['N','S','V','F'])[2]


[[43237   315   180   288]
 [ 1708   100   231    10]
 [  734    99  2281   106]
 [  376     0     8     4]]
              precision    recall  f1-score   support

           N       0.94      0.98      0.96     44020
           S       0.19      0.05      0.08      2049
           V       0.84      0.71      0.77      3220
           F       0.01      0.01      0.01       388

   micro avg       0.92      0.92      0.92     49677
   macro avg       0.50      0.44      0.45     49677
weighted avg       0.89      0.92      0.90     49677



array(['N', 'N', 'N', ..., 'N', 'N', 'N'], dtype='<U1')